In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import scipy

### All the lap times have been picked up from Wikipedia and car specs from fastestlaps.com 

![Track](images/NurburgringNordschielfe.jpg)

In [2]:
df = pd.read_csv("C:\\Users\\dell\\Desktop\\Python Programs\\MiniPrject\\CSVfiles\\nurburgringCarData.csv")
df.head()

,Car,Lap time,PS,KG,0-100,0-200,0-300,top speed,Est. downforce @ 200 kph,Est. downforce @ 250 kph,Est. downforce @ 300 kph,Drivetrain,Powertrain,Aspiration
0,Mercedes - AMG One,6.29.09,1063,1620,2.7,7,15.6,352,752.0,1000.0,1352.0,AWD,Hybrid,Turbocharged
1,Porsche 911 GT2 RS Manthey Performance Kit (991),6.40.30,700,1420,2.6,7.8,21.5,343,469.0,621.0,806.0,RWD,ICE,Turbocharged
2,AMG GT Black Series,6.48.05,730,1637,2.8,8.3,26.7,352,449.0,600.0,740.0,RWD,ICE,Turbocharged
3,Corvette ZR1X ZTK Package,6.49.27,1267,1794,2.2,5.8,12.4,225 mph,381.0,484.0,608.0,AWD,Hybrid,Turbocharged
4,Porsche 911 GT3 RS (992) specs,6.49.30,585,1480,2.6,8.3,NaN,296,509.0,785.0,988.0,RWD,ICE,Natural Aspiration


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Car                       54 non-null     object 
 1   Lap time                  54 non-null     object 
 2   PS                        54 non-null     int64  
 3   KG                        54 non-null     int64  
 4   0-100                     53 non-null     object 
 5   0-200                     52 non-null     object 
 6   0-300                     31 non-null     object 
 7   top speed                 54 non-null     object 
 8   Est. downforce @ 200 kph  51 non-null     float64
 9   Est. downforce @ 250 kph  51 non-null     float64
 10  Est. downforce @ 300 kph  50 non-null     float64
 11  Drivetrain                54 non-null     object 
 12  Powertrain                54 non-null     object 
 13  Aspiration                54 non-null     object 
dtypes: float64(3

### Converting lap times to just seconds

In [4]:
def laptimeSec(x):
    List = x.split(".")
    seconds = 60*float(List[0]) + float(List[1]) + 0.01*float(List[2])
    return seconds


df["Lap time"] = df["Lap time"].apply(lambda x: laptimeSec(x))

### Adding a row for power in horsepower as well as kw (hp is more used and kw is best for calculations)

In [5]:
def pstoKw (x): 
    return x*0.7355

def pstphp (x): 
    return x*0.98632

df["HP"] = df["PS"].apply(lambda x: pstoKw(x))
df["KW"] = df["PS"].apply(lambda x: pstphp(x))

#df 

### Now ill be converting all the mph to kmph
### Top Speed cpnversion is straight forward
### Acceleration follows saturation function so v(t) = Top speed (1 - e^(-kt)) 
### t(v) = inverse of v(t) which gives: -1/k * ln (1 - v/top speed) where k is like the rate constant
### Note: Here we are treating time as dependant because we need to find times for acceleration

In [6]:
def MphToKmph(x):
    return x*1.60934

def topSpeed(y):
    if pd.isna(y):
        return np.nan
    x = str(y)
    List = x.split(" ")
    if "mph" in List:
        return MphToKmph(float(List[0]))
    else:
        return float(List[0])
    
df["top speed"] = df["top speed"].apply(lambda x: topSpeed(x))
#df
df1 = df.copy()
#df1 

### Checking NaN values

In [7]:
nan_df = df[df.isna().any(axis =1 )]
#nan_df

### here we will use 1 / 1 - (v / topspeed)**p) because for top speeds close to 300, the time is blowing up

In [8]:
def t_of_v(v, A, p, topspeed):
    return A * np.log(1 / (1 - (v / topspeed) ** p))

def best_params(v, t, topspeed, p0=[1.0, 1.0]):
    params, cov = scipy.optimize.curve_fit(lambda v, A, p: t_of_v(v, A, p, topspeed), v, t, p0=p0, maxfev=5000 )
    return params 

In [9]:
def accelcorrector(y):
    x = y.copy()
    List = [x["0-100"], x["0-200"], x["0-300"]]
    if pd.isna(List[0]):
        return x  
    
    
    popped = False 
    if pd.isna(List[2]):
        List.pop(2)
        popped = True

    speeds = [] 
    times = [] 

    for i in List: 
        j = str(i)
        temp = j.split(" ")
        if "mph" in temp:
            mph_val = float(temp[-1])
            speeds.append(MphToKmph(mph_val))
            times.append(float(temp[0]))
        else: 
            times.append(float(temp[0]))
    
    
    if speeds:
        v = np.array(speeds, dtype=float)
        t = np.array(times, dtype=float)
        topspeed = float(x["top speed"])
        requiredV = np.array([100, 200, 300], dtype=float)
        
        
        A, p = best_params(v, t, topspeed)
        predicted_t = t_of_v(requiredV, A, p, topspeed)

        
        x["0-100"] = predicted_t[0]
        x["0-200"] = predicted_t[1]
        x["0-300"] = np.nan if popped else predicted_t[2]
        
        
        if popped:
            x["pred_0-300"] = predicted_t[2]
        else:
            x["pred_0-300"] = x["0-300"]
    else:
        
        x["pred_0-300"] = np.nan if popped else times[2]

    return x


In [10]:
df = df.apply(accelcorrector , axis = 1)
#df

C:\Users\dell\AppData\Local\Temp\ipykernel_11280\3503716279.py:2: RuntimeWarning: invalid value encountered in log
  return A * np.log(1 / (1 - (v / topspeed) ** p))
C:\Users\dell\AppData\Local\Temp\ipykernel_11280\3503716279.py:2: RuntimeWarning: invalid value encountered in log
  return A * np.log(1 / (1 - (v / topspeed) ** p))
C:\Users\dell\AppData\Local\Temp\ipykernel_11280\3503716279.py:5: OptimizeWarning: Covariance of the parameters could not be estimated
  params, cov = scipy.optimize.curve_fit(lambda v, A, p: t_of_v(v, A, p, topspeed), v, t, p0=p0, maxfev=5000 )
C:\Users\dell\AppData\Local\Temp\ipykernel_11280\3503716279.py:2: RuntimeWarning: invalid value encountered in log
  return A * np.log(1 / (1 - (v / topspeed) ** p))


In [11]:
df.head(7)

,0-100,0-200,0-300,Aspiration,Car,Drivetrain,Est. downforce @ 200 kph,Est. downforce @ 250 kph,Est. downforce @ 300 kph,HP,KG,KW,Lap time,PS,Powertrain,pred_0-300,top speed
0,2.7,7,15.6,Turbocharged,Mercedes - AMG One,AWD,752.0,1000.0,1352.0,781.8365,1620,1048.45816,389.09,1063,Hybrid,15.600000,352.0000
1,2.6,7.8,21.5,Turbocharged,Porsche 911 GT2 RS Manthey Performance Kit (991),RWD,469.0,621.0,806.0,514.8500,1420,690.42400,400.30,700,ICE,21.500000,343.0000
2,2.8,8.3,26.7,Turbocharged,AMG GT Black Series,RWD,449.0,600.0,740.0,536.9150,1637,720.01360,408.05,730,ICE,26.700000,352.0000
3,2.2,5.8,12.4,Turbocharged,Corvette ZR1X ZTK Package,AWD,381.0,484.0,608.0,931.8785,1794,1249.66744,409.27,1267,Hybrid,12.400000,362.1015
4,2.6,8.3,NaN,Natural Aspiration,Porsche 911 GT3 RS (992) specs,RWD,509.0,785.0,988.0,430.2675,1480,576.99720,409.30,585,ICE,NaN,296.0000
5,1.938598,6.636789,17.792403,Turbocharged,Corvette ZR1 ZTK Package,RWD,281.0,426.0,608.0,793.6045,1701,1064.23928,410.76,1079,ICE,17.792403,362.0000
6,2.712191,9.45539,31.044418,Supercharged,Ford Mustand GTD,RWD,422.0,659.0,949.0,598.6970,1772,802.86448,412.07,814,ICE,31.044418,325.0000


In [12]:
df1.head(7)

,Car,Lap time,PS,KG,0-100,0-200,0-300,top speed,Est. downforce @ 200 kph,Est. downforce @ 250 kph,Est. downforce @ 300 kph,Drivetrain,Powertrain,Aspiration,HP,KW
0,Mercedes - AMG One,389.09,1063,1620,2.7,7,15.6,352.0000,752.0,1000.0,1352.0,AWD,Hybrid,Turbocharged,781.8365,1048.45816
1,Porsche 911 GT2 RS Manthey Performance Kit (991),400.30,700,1420,2.6,7.8,21.5,343.0000,469.0,621.0,806.0,RWD,ICE,Turbocharged,514.8500,690.42400
2,AMG GT Black Series,408.05,730,1637,2.8,8.3,26.7,352.0000,449.0,600.0,740.0,RWD,ICE,Turbocharged,536.9150,720.01360
3,Corvette ZR1X ZTK Package,409.27,1267,1794,2.2,5.8,12.4,362.1015,381.0,484.0,608.0,AWD,Hybrid,Turbocharged,931.8785,1249.66744
4,Porsche 911 GT3 RS (992) specs,409.30,585,1480,2.6,8.3,NaN,296.0000,509.0,785.0,988.0,RWD,ICE,Natural Aspiration,430.2675,576.99720
5,Corvette ZR1 ZTK Package,410.76,1079,1701,2.2 mph 60,5.9 mph 120,16.0 mph 180,362.0000,281.0,426.0,608.0,RWD,ICE,Turbocharged,793.6045,1064.23928
6,Ford Mustand GTD,412.07,814,1772,2.8 mph 60,10.2 mph 130,21.2 mph 170,325.0000,422.0,659.0,949.0,RWD,ICE,Supercharged,598.6970,802.86448


In [13]:
df = df1.copy()

### There still seems to be a significant difference in acceleration, ill manually correct them with approximations. The above formula will work better to predict intermediate timings, such as 0-250, 0-150, 0-175 etc. It wont work for extremes.

In [14]:
def accelcorrector2(y):
    mults = {
        "60": 1.005,
        "120": 1.01, 
        "130": 0.99,
        "170": 1.075,
        "180": 1.05
    }

    x = y.copy()
    List = [x["0-100"], x["0-200"], x["0-300"]]

    if pd.isna(List[0]):
        return x
    
    popped = False 
    if pd.isna(List[2]):
        List.pop(2)
        popped = True

    speeds = [] 
    times = [] 
    m = []

    for i in List: 
        j = str(i)
        temp = j.split(" ")

        if "mph" in temp:
            mph_val = temp[-1]
            m.append(mults.get(mph_val, 1.0))
            temp[-1] = MphToKmph(float(temp[-1]))
            speeds.append(temp[-1])
            times.append(float(temp[0]))
        else: 
            times.append(float(temp[0]))
            m.append(1.0)

    predictedt = [times[i]*m[i] for i in range(len(times))]

    x["0-100"] = predictedt[0]
    x["0-200"] = predictedt[1]
    x["0-300"] = np.nan if popped else predictedt[2]

    return x


In [15]:
df = df.apply(accelcorrector2 , axis = 1)
#df

In [16]:
df.head(7)

,Car,Lap time,PS,KG,0-100,0-200,0-300,top speed,Est. downforce @ 200 kph,Est. downforce @ 250 kph,Est. downforce @ 300 kph,Drivetrain,Powertrain,Aspiration,HP,KW
0,Mercedes - AMG One,389.09,1063,1620,2.700,7.000,15.60,352.0000,752.0,1000.0,1352.0,AWD,Hybrid,Turbocharged,781.8365,1048.45816
1,Porsche 911 GT2 RS Manthey Performance Kit (991),400.30,700,1420,2.600,7.800,21.50,343.0000,469.0,621.0,806.0,RWD,ICE,Turbocharged,514.8500,690.42400
2,AMG GT Black Series,408.05,730,1637,2.800,8.300,26.70,352.0000,449.0,600.0,740.0,RWD,ICE,Turbocharged,536.9150,720.01360
3,Corvette ZR1X ZTK Package,409.27,1267,1794,2.200,5.800,12.40,362.1015,381.0,484.0,608.0,AWD,Hybrid,Turbocharged,931.8785,1249.66744
4,Porsche 911 GT3 RS (992) specs,409.30,585,1480,2.600,8.300,NaN,296.0000,509.0,785.0,988.0,RWD,ICE,Natural Aspiration,430.2675,576.99720
5,Corvette ZR1 ZTK Package,410.76,1079,1701,2.211,5.959,16.80,362.0000,281.0,426.0,608.0,RWD,ICE,Turbocharged,793.6045,1064.23928
6,Ford Mustand GTD,412.07,814,1772,2.814,10.098,22.79,325.0000,422.0,659.0,949.0,RWD,ICE,Supercharged,598.6970,802.86448


In [17]:
df1.head(7)

,Car,Lap time,PS,KG,0-100,0-200,0-300,top speed,Est. downforce @ 200 kph,Est. downforce @ 250 kph,Est. downforce @ 300 kph,Drivetrain,Powertrain,Aspiration,HP,KW
0,Mercedes - AMG One,389.09,1063,1620,2.7,7,15.6,352.0000,752.0,1000.0,1352.0,AWD,Hybrid,Turbocharged,781.8365,1048.45816
1,Porsche 911 GT2 RS Manthey Performance Kit (991),400.30,700,1420,2.6,7.8,21.5,343.0000,469.0,621.0,806.0,RWD,ICE,Turbocharged,514.8500,690.42400
2,AMG GT Black Series,408.05,730,1637,2.8,8.3,26.7,352.0000,449.0,600.0,740.0,RWD,ICE,Turbocharged,536.9150,720.01360
3,Corvette ZR1X ZTK Package,409.27,1267,1794,2.2,5.8,12.4,362.1015,381.0,484.0,608.0,AWD,Hybrid,Turbocharged,931.8785,1249.66744
4,Porsche 911 GT3 RS (992) specs,409.30,585,1480,2.6,8.3,NaN,296.0000,509.0,785.0,988.0,RWD,ICE,Natural Aspiration,430.2675,576.99720
5,Corvette ZR1 ZTK Package,410.76,1079,1701,2.2 mph 60,5.9 mph 120,16.0 mph 180,362.0000,281.0,426.0,608.0,RWD,ICE,Turbocharged,793.6045,1064.23928
6,Ford Mustand GTD,412.07,814,1772,2.8 mph 60,10.2 mph 130,21.2 mph 170,325.0000,422.0,659.0,949.0,RWD,ICE,Supercharged,598.6970,802.86448


### Adding power to weight and downforce to weight ratios

In [18]:
df["Power/Weight"] = df["KW"]/df["KG"]
df["downforce 250 / weight"] = df["Est. downforce @ 250 kph"] / df["KG"]
df["downforce 200 / weight"] = df["Est. downforce @ 200 kph"] / df["KG"]
#df

### Now ill be dealing NaNs in columns like 0 - 200,300 and downforce at 200,250,300 for cars having top speed less than 200 250 and 300

In [19]:
df["0-200"] = np.where(df["top speed"] < 200 , np.inf , df["0-200"]) 
df["Est. downforce @ 200 kph"] = np.where(df["top speed"] < 200 , 0 , df["Est. downforce @ 200 kph"]) 
df["0-300"] = np.where(df["top speed"] < 300 , np.inf , df["0-300"]) 
df["Est. downforce @ 300 kph"] = np.where(df["top speed"] < 300 , 0 , df["Est. downforce @ 300 kph"]) 
df["Est. downforce @ 250 kph"] = np.where(df["top speed"] < 250 , 0 , df["Est. downforce @ 250 kph"]) 
#df

### Sorting by laptimes

In [20]:
df.sort_values(by = "Lap time" , inplace=True)
#df

### Checking rows with no NaNs

In [21]:
Notnan_df = df.dropna(how = "any")
Notnan_df.sort_values(by = "Lap time")
#Notnan_df

,Car,Lap time,PS,KG,0-100,0-200,0-300,top speed,Est. downforce @ 200 kph,Est. downforce @ 250 kph,Est. downforce @ 300 kph,Drivetrain,Powertrain,Aspiration,HP,KW,Power/Weight,downforce 250 / weight,downforce 200 / weight
31,Porsche 919 Hybrid Evo,319.55,1160,829,1.800,4.100,8.5000,369.0000,900.0,1600.0,2400.0,AWD,Hybrid,Turbocharged,853.1800,1144.13120,1.380134,1.930036,1.085645
32,Volkswagen IDR,365.36,680,1100,2.250,4.500,inf,270.0000,650.0,1200.0,0.0,AWD,Electric,Electric,500.1400,670.69760,0.609725,1.090909,0.590909
0,Mercedes - AMG One,389.09,1063,1620,2.700,7.000,15.6000,352.0000,752.0,1000.0,1352.0,AWD,Hybrid,Turbocharged,781.8365,1048.45816,0.647196,0.617284,0.464198
1,Porsche 911 GT2 RS Manthey Performance Kit (991),400.30,700,1420,2.600,7.800,21.5000,343.0000,469.0,621.0,806.0,RWD,ICE,Turbocharged,514.8500,690.42400,0.486214,0.437324,0.330282
2,AMG GT Black Series,408.05,730,1637,2.800,8.300,26.7000,352.0000,449.0,600.0,740.0,RWD,ICE,Turbocharged,536.9150,720.01360,0.439837,0.366524,0.274282
3,Corvette ZR1X ZTK Package,409.27,1267,1794,2.200,5.800,12.4000,362.1015,381.0,484.0,608.0,AWD,Hybrid,Turbocharged,931.8785,1249.66744,0.696582,0.269788,0.212375
4,Porsche 911 GT3 RS (992) specs,409.30,585,1480,2.600,8.300,inf,296.0000,509.0,785.0,0.0,RWD,ICE,Natural Aspiration,430.2675,576.99720,0.389863,0.530405,0.343919
5,Corvette ZR1 ZTK Package,410.76,1079,1701,2.211,5.959,16.8000,362.0000,281.0,426.0,608.0,RWD,ICE,Turbocharged,793.6045,1064.23928,0.625655,0.250441,0.165197
6,Ford Mustand GTD,412.07,814,1772,2.814,10.098,22.7900,325.0000,422.0,659.0,949.0,RWD,ICE,Supercharged,598.6970,802.86448,0.453084,0.371896,0.238149
7,Porsche 911 GT3 RS Manthey Performance Kit (99...,413.34,560,1400,2.700,9.200,inf,285.0000,355.0,555.0,0.0,RWD,ICE,Natural Aspiration,411.8800,552.33920,0.394528,0.396429,0.253571


In [22]:
df.describe()

c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:4653: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,Lap time,PS,KG,0-100,0-200,0-300,top speed,Est. downforce @ 200 kph,Est. downforce @ 250 kph,Est. downforce @ 300 kph,HP,KW,Power/Weight,downforce 250 / weight,downforce 200 / weight
count,54.000000,54.000000,54.000000,53.000000,53.000,52.0,54.000000,51.000000,51.000000,51.000000,54.000000,54.000000,54.000000,51.000000,51.000000
mean,443.144074,716.944444,1758.185185,3.358264,inf,inf,307.853731,167.196078,251.764706,276.392157,527.312639,707.136644,0.407357,0.181257,0.117742
std,35.551354,434.637711,384.663983,1.030279,NaN,NaN,49.397849,216.899426,343.598579,431.451971,319.676037,428.691868,0.220572,0.325512,0.191808
min,319.550000,300.000000,829.000000,1.800000,4.100,8.5,199.000000,0.000000,0.000000,0.000000,220.650000,295.896000,0.178681,0.000000,0.000000
25%,419.352500,502.500000,1447.750000,2.700000,8.787,23.5,270.500000,0.000000,0.000000,0.000000,369.588750,495.625800,0.298629,0.000000,0.000000
50%,447.005000,600.000000,1664.000000,3.200000,10.600,31.3,315.000000,70.000000,120.000000,120.000000,441.300000,591.792000,0.339204,0.061842,0.037493
75%,467.980000,695.000000,2145.750000,3.800000,15.500,NaN,332.000000,290.500000,436.500000,458.500000,511.172500,685.492400,0.409800,0.260701,0.171833
max,523.000000,3018.000000,2655.000000,5.800000,inf,inf,496.000000,900.000000,1600.000000,2400.000000,2219.739000,2976.713760,1.380134,1.930036,1.085645


In [23]:
Notnan_df.describe()

c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:4653: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,Lap time,PS,KG,0-100,0-200,0-300,top speed,Est. downforce @ 200 kph,Est. downforce @ 250 kph,Est. downforce @ 300 kph,HP,KW,Power/Weight,downforce 250 / weight,downforce 200 / weight
count,50.000000,50.000000,50.000000,50.000000,50.000,50.00,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,444.678000,659.420000,1735.800000,3.401500,inf,inf,302.802030,163.480000,248.060000,270.260000,485.003410,650.399134,0.388977,0.179573,0.115808
std,36.504456,279.007724,375.605144,1.044124,NaN,NaN,43.059055,217.455343,346.056525,433.581469,205.210181,275.190898,0.195045,0.328592,0.193252
min,319.550000,300.000000,829.000000,1.800000,4.100,8.50,199.000000,0.000000,0.000000,0.000000,220.650000,295.896000,0.178681,0.000000,0.000000
25%,418.117500,502.500000,1431.750000,2.700000,9.250,23.55,266.250000,0.000000,0.000000,0.000000,369.588750,495.625800,0.295223,0.000000,0.000000
50%,448.500000,595.000000,1653.500000,3.250000,10.600,31.75,313.000000,67.500000,110.000000,66.000000,437.622500,586.860400,0.335560,0.057390,0.035988
75%,468.957500,677.500000,2098.000000,3.875000,16.775,NaN,329.000000,276.750000,433.500000,454.000000,498.301250,668.231800,0.401900,0.256911,0.164019
max,523.000000,1941.000000,2655.000000,5.800000,inf,inf,415.000000,900.000000,1600.000000,2400.000000,1427.605500,1914.447120,1.380134,1.930036,1.085645


In [24]:
df.to_csv("CleanedData.csv")
Notnan_df.to_csv("CleanedData_withNoNaN.csv")